In [2]:
%run Requirements.ipynb

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/13 16:54:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Artist Details and Images

In [ ]:
df = spark.sql("""SELECT DISTINCT Artist FROM PARQUET.`/Users/ronitguptaaa/Documents/Music-Fusion-APP/Mart/MusicFusion-Details` ORDER BY Artist""")

In [21]:
auth_header = base64.b64encode((spotify_secrets.get("client_id")+':'+spotify_secrets.get("client_secret")).encode()).decode()
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Authorization": f"Basic {auth_header}"
}
data = {
    "grant_type": "client_credentials"
}

response = requests.post(spotify_auth_url, headers=headers, data=data)
response_data = response.json()['access_token']

headers = {
    "Authorization": f"Bearer {response_data}"
}

response = requests.get(spotify_url, headers=headers)
spotify_response_data=response.json()

In [22]:
artist_names = [row['Artist'] for row in df.collect()]

artist_details = []

for artist_name in artist_names:
    spotify_api_url = f'https://api.spotify.com/v1/search?q={artist_name}&type=artist'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    try:
        response = requests.get(spotify_api_url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            artists = data.get('artists', {}).get('items', [])

            for artist_data in artists:
                if artist_data.get('name').lower() == artist_name.lower():
                    artist_details.append({
                        'name': artist_data.get('name'),
                        'id': artist_data.get('id'),
                        'image_url': artist_data.get('images')[0].get('url') if artist_data.get('images') else None,
                        'followers': artist_data.get('followers', {}).get('total'),
                        'popularity': artist_data.get('popularity')
                    })
                    break

    except requests.exceptions.RequestException as e:
        print(f"Request failed for artist '{artist_name}': {e}")

artist_df = spark.createDataFrame(artist_details)

artist_df.show(truncate=False)

+---------+----------------------+----------------------------------------------------------------+-----------------+----------+
|followers|id                    |image_url                                                       |name             |popularity|
+---------+----------------------+----------------------------------------------------------------+-----------------+----------+
|2742433  |22wbnEMDvgVIAGdFeek6ET|https://i.scdn.co/image/ab6761610000e5ebb2ce21a89c09c00d80d8ca25|Benson Boone     |85        |
|95208705 |6qqNVTkY8uBg9cP3Jd7DAH|https://i.scdn.co/image/ab6761610000e5eb4a21b4760d2ecb7b0dcdc8da|Billie Eilish    |94        |
|1084710  |7GlBOeep6PqTfFi59PTUUN|https://i.scdn.co/image/ab6761610000e5ebcde5a0d57c1b79de5fce6bee|Chappell Roan    |81        |
|1380537  |5p9HO3XC5P3BLxJs5Mtrhm|https://i.scdn.co/image/ab6761610000e5eb02fcc327683d53bea9583c69|Djo              |77        |
|85281135 |7dGJo4pcD2V6oG8kP0tJRR|https://i.scdn.co/image/ab6761610000e5eba00b11c129b27a88fc72f36